# LLM from scratch 

We follow this [tutorial](https://medium.com/@msouza.os/llm-from-scratch-with-pytorch-9f21808c6319)

In [165]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import re

## Tokenization

**Idea**: need to make the machine able to read text. Since it can only understand numbers, need to _encode text into numbers_. Since humans can only understand text, we need to be able to _decode numbers into text_

=> need numeric representation of text 

1. split text into sequence of words (_tokens_)
2. for each word (_token_) in model vocabulrary, assign integer

There are several different models for tokenizers. Note that model performance during training will be affected by chosen tokenizer.

In [ ]:
class Tokenizer:

    @staticmethod
    def create_vocab(dataset:str)->dict[str:str]:
        """
        Create vocabulary

        :param dataset: Text data to be used to define vocabulary
        :type dataset: str

        :returns: (word) vocabulary  
        :rtype: dict[str:int]
        """

        # CREATE VOCABULRARY
        pattern_ = re.compile(r'(?<=\w)(?=[^\w])|(?<=[^\w])(?=\w)|(?<=[^\w])(?=\s)|(?<=[^\w])(?=[^w])') # pattern splits string into words, spaces and special characters
        words_in_data = pattern_.split(dataset)
        vocab = {tkn:idx for idx,tkn in enumerate(sorted(set(words_in_data)))}

        # ADD UNKNOWN TOKEN
        vocab['<ukn>'] = len(vocab)

        return vocab
    

    def __init__(self,vocab:dict[str,int]) -> None:
        """
        Initialize tokenizer

        :param vocab: vocabulary to intitalize
        :type vocab: dict[str:int]
        """
        self.vocab_encode = {str(k): int(v) for k, v in vocab.items()} # encode vocab: key:value = char:int
        self.vocab_decode = {v: k for k, v in self.vocab_encode.items()} # decode vocab key:value = int:char
        self.pattern = re.compile(r'(?<=\w)(?=[^\w])|(?<=[^\w])(?=\w)|(?<=[^\w])(?=\s)|(?<=[^\w])(?=[^w])')

    def encode(self,text:str):
        """
        Encode text in (lvl: character)

        :param text: Input text to encode
        :type text: str

        :returns: encoded text (list with token indices)
        :rtype: list[int]
        """

        text_ = self.pattern.split(text)
        encoded_txt = [self.vocab_encode.get(word,self.vocab_encode['<ukn>']) for word in text_]
        return encoded_txt
    
    def decode(self,idxs:list[int]):
        """
        Decode a list of token indices

        :param idxs: List of token indices
        :type idxs: list[int]

        :returns: Decoded text
        :rtype: str
        """

        decoded_txt = [self.vocab_decode.get(idx,'<ukn>') for idx in idxs]

        return ''.join(decoded_txt)

### Sanity Check

In [161]:
dataset = 'Hallo Welt! Wie geht es Dir?'
vocab = Tokenizer.create_vocab(dataset)

tokenize = Tokenizer(vocab)

test = 'Hallo Welt :)'

enc = tokenize.encode(test)
dec = tokenize.decode(enc)

print(f'{enc = }\n')
print(f'{dec = }')

enc = [4, 0, 5, 0, 9, 9]

dec = 'Hallo Welt <ukn><ukn>'


## Embedding 

**Idea:** We want the computer to understand the symmantics (not only syntax). Thus we need to teach it what words (characters) are _similar_. To do so, we use vector represetnations (**embeddings**) of the tokens. These embeddings come with learnable parameters that can be adjusted during training to imporve the representation. The embedding is thus a map from the vocabulary $V$ to some vector space $X$: $E \colon V \to X$. If $\dim(V) = N$ (length of vodabulary) and $\dim(X) = d$, then the embedding is a _linear_ map $E \colon \mathbb{R}^N \to \mathbb{R}^d$ so a $N \times d$ matrix. 

Notice that the larger $d$, the more parameters can be learned and hence the more the model can learn more details about the dataset at the expense of computation time.


**Implementation:** 

1. Assign to each token a random vector of parameters (which will be learned during training). Assume that the length of the vocabulary is $N$, and we embedd into a vector space of dimension $d$, the embedding space will be $\mathbb{R}^N \times \mathbb{R}^d$ which can be represented as a matrix with $N$ rows and $d$ columns.

2. To compute similarity, use _cosine_ distance: for $x,y \in \mathbb{R}^d$, define their cosine-distance by $d(x,y) = \cos( \sphericalangle (x,y) ) \in [-1,1]$

Note that 

$$ \cos( \sphericalangle (x,y) ) = \frac{\langle x, y \rangle}{\lVert x \rVert \lVert y \rVert}$$ 

## Transformers

**!!** LLMs = Transformer based models

**Transformer model architecture:** [Attention is all you need](https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)

1. composed of 1 _encoder_ and 1 _decoder_ block (using attention mechanism called _Scaled Dot-Product Attention_)

2. How they work
    * Encoder block: tries to analyze each word considering the entire text context
    * Decoder block: masks all future words at some specific position and uses only the _previous_ words available to analyze the current one
    * Main block: _Multi-Head Attention_ (!! key component of LLMs)

## Attention Mechanism 

### Scaled Dot-Product Attention

**Input:**
* Q: query = information we're trying to find (relationship between tokens)
* K: Key =  where you will find this information
* V: Value



**architecture:** 

```
Q --> |
      | MatMult --> Scale --> Mask(opt.) --> SoftMax --> |
K --> |                                                  |   
                                                         | -- MatMult -->
                                                         |
V ------------------------------------------------------>|
```


$Q,K,V$ are calculated by linear transformations (simple linear layers) from the embedded text. The workflow is roughly as follows: for $T \in Mat(\mathbb{R},N\times N)$

$\text{text (str)} \stackrel{E\;(\text{embedding})}{\longrightarrow} Q,K \in \mathbb{R}^N \times \mathbb{R}^d \stackrel{T}{\longrightarrow} Q',K' \in \mathbb{R}^N \times \mathbb{R}^d  \stackrel{Q K^t}{\longrightarrow} R \in \mathbb{R}^N \times \mathbb{R}^N$ 

The output of this sequence is a $N\times N$ matrix $R$ encoding the _relationship_ between our tokens. 

Next, we need to scale (to avoid 1-hot encoding) by $\sqrt{N}$ ($N$ is called the _embedding dimension_) and then feed it into a SoftMax function to normalize each value to (0,1) such that each row sums up to 1. This defines the so-called _attention_

$$A = \frac{Q K^t}{\sqrt{N}}$$

Finally, we need to multiply it with the $V$ to obtain a "vector representation" if the information we were looking for:

$$ \text{out} = A V$$ 

_Remark_: It seems to be custom to scale A by the _embedding size_ $d$

_NOTE_:  The attention mechanisim for transformers is trying to predict **every single token** in our sequence and not only the last one.
However, during training we actually want to predict only the next token given a sequence of tokens as _context_. We could even look at a Markov Chain by considering only the previous token to predict the next one. The problem is that the output $out = A V$ considers _all_ possible tokens: the entire attention matrix $A$ is used.
To overcome this problem, we mask $A$. We can multiply $A$ simply by a lower triangular matrix $M$ with all non-zero entries being 1. The mask then hides all consecutive tokens of all samples.





In [237]:
# IMPLEMENTATION
torch.random.manual_seed(seed=42)


# DATA
text = "Hi! I am Donald."
vocab = Tokenizer.create_vocab(text)
tokenize = Tokenizer(vocab)

tokens = tokenize.encode(text) # [4, 1, 0, 5, 0, 6, 0, 3, 2]
tokens.reverse()

# PARAMETERS
# vocab_size = max(tokens) + 1 # numbers of tokens (classes) to predict 
vocab_size = max(vocab.values())  #N = numbers of tokens (classes) to predict (last class is <ukn> but 0 based)
emb_dim = 3 # d = size of vecrep of each token
context = len(tokens) # context size of model

# LAYERS
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim) # tokens = (N,) -> (N,3)
query = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False) # linear transformation on 2nd idx (N,3) -> (N,3) @ (3,3) = (N,3)
key = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False)
value = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False)

# MASK
ones = torch.ones(size=[context, context], dtype=torch.float)
mask = torch.tril(input=ones) # lower triangular matrix 

# TOKEN IDXS FOR POS EMBEDDING
tkn_idxs = torch.arange(context, dtype=torch.long)

# FORWARD PASS
t_tokens = torch.tensor(tokens).unsqueeze(dim=0) # (N,) -> (1,N)

x = embedding(t_tokens) # (1,N) -> (1,N,d) embedded vectors

B, T, C = x.size()

# apply linear transformation T
Q = query(x) # (1,N,d) -> (1,N,d) @ (1,d,d) = (1,N,d)
K = key(x) 
V = value(x) 

# matmul
QK = Q @ K.transpose(-2, -1) # (1,N,d) @ (1,d,N) -> (1,N,N)
A = QK  * C**-0.5 # attention matrix 
A.masked_fill_(mask[:T,:T] == 0, float("-inf")) # applying mask
A = F.softmax(input=A, dim=-1) # apply SoftMax: (1,N,N) normalizing to 0 and 1 in embedding dimension

out = A @ V # (1,N,N) @ (1,N,d) -> (1,N,d)

# print(f'{out = }\n') # new data representation
print(f'{out.size() = }\n') 

# x_before = x.detach().clone()
x_after = x.detach().clone()


out.size() = torch.Size([1, 9, 3])



In [235]:
print('x before reversing the order of the token sequence')
print(x_before)

x before reversing the order of the token sequence
tensor([[[-1.1109,  0.0418, -0.2516],
         [-2.1055,  0.6784,  1.0783],
         [ 1.9269,  1.4873,  0.9007],
         [ 0.8599, -0.3097, -0.3957],
         [ 1.9269,  1.4873,  0.9007],
         [ 0.8034, -0.6216, -0.5920],
         [ 1.9269,  1.4873,  0.9007],
         [-0.6866, -0.4934,  0.2415],
         [ 0.8008,  1.6806,  0.3559]]])


In [238]:
print('x after reversing the order of the token sequence')
print(x_after)

x after reversing the order of the token sequence
tensor([[[ 0.8008,  1.6806,  0.3559],
         [-0.6866, -0.4934,  0.2415],
         [ 1.9269,  1.4873,  0.9007],
         [ 0.8034, -0.6216, -0.5920],
         [ 1.9269,  1.4873,  0.9007],
         [ 0.8599, -0.3097, -0.3957],
         [ 1.9269,  1.4873,  0.9007],
         [-2.1055,  0.6784,  1.0783],
         [-1.1109,  0.0418, -0.2516]]])


We see that the entries in x _before_ and _after_ reversing the order of the input sequence of tokens are essentially the same up to orientation. 
The columns (same for rows) contain the same numbers but in reversed order, as seen below

In [271]:
x_before_col1 = [elem for elem in x_before[0][:,0]]
x_after_col1 = [elem for elem in x_after[0][:,0]]
x_after_col1.reverse()

print(f'before: {"after:":>15} {"diff:":>14}')
for i in range(len(x_before_col1)):
    print(f'{x_before_col1[i]: .3f} {x_after_col1[i]: >15.3f} {x_before_col1[i] - x_after_col1[i]: >15.3f}')


before:          after:          diff:
-1.111          -1.111           0.000
-2.106          -2.106           0.000
 1.927           1.927           0.000
 0.860           0.860           0.000
 1.927           1.927           0.000
 0.803           0.803           0.000
 1.927           1.927           0.000
-0.687          -0.687           0.000
 0.801           0.801           0.000


## Positional Encoding

Location of words in sentences matter.
The vector representation should thereofre depend on the order of the token sequence.

However, the attention mechanism above considers every token all at once in order to get a sense of _context_. Hence, we need to encode the positions by hand. This can be done by introducing another set of _positional_ parameters which can be learned. We simply define yet another embedding (of dimension context x emb_dim) and add it to our token embedding. 

In [273]:
# IMPLEMENTATION
torch.random.manual_seed(seed=42)


# DATA
text = "Hi! I am Donald."
vocab = Tokenizer.create_vocab(text)
tokenize = Tokenizer(vocab)

tokens = tokenize.encode(text) # [4, 1, 0, 5, 0, 6, 0, 3, 2]
tokens.reverse()

# PARAMETERS
# vocab_size = max(tokens) + 1 # numbers of tokens (classes) to predict 
vocab_size = max(vocab.values())  #N = numbers of tokens (classes) to predict (last class is <ukn> but 0 based)
emb_dim = 3 # d = size of vecrep of each token
context = len(tokens) # context size of model

# LAYERS
pos_emb = nn.Embedding(num_embeddings=context, embedding_dim=emb_dim) # !! positional encoding 
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim) # tokens = (N,) -> (N,3)
query = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False) # linear transformation on 2nd idx (N,3) -> (N,3) @ (3,3) = (N,3)
key = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False)
value = nn.Linear(in_features=emb_dim, out_features=emb_dim, bias=False)

# MASK
ones = torch.ones(size=[context, context], dtype=torch.float)
mask = torch.tril(input=ones) # lower triangular matrix 

# TOKEN IDXS FOR POS EMBEDDING
tkn_idxs = torch.arange(context, dtype=torch.long)

# FORWARD PASS
t_tokens = torch.tensor(tokens).unsqueeze(dim=0) # (N,) -> (1,N)

x = embedding(t_tokens) # (1,N) -> (1,N,d) embedded vectors
x += pos_emb(tkn_idxs) # !! add pos embeddings

B, T, C = x.size()

# apply linear transformation T
Q = query(x) # (1,N,d) -> (1,N,d) @ (1,d,d) = (1,N,d)
K = key(x) 
V = value(x) 

# matmul
QK = Q @ K.transpose(-2, -1) # (1,N,d) @ (1,d,N) -> (1,N,N)
A = QK  * C**-0.5 # attention matrix 
A.masked_fill_(mask[:T,:T] == 0, float("-inf")) # applying mask
A = F.softmax(input=A, dim=-1) # apply SoftMax: (1,N,N) normalizing to 0 and 1 in embedding dimension

out = A @ V # (1,N,N) @ (1,N,d) -> (1,N,d)

print(f'{out = }\n') # new data representation
print(f'{out.size() = }\n') 

# x_before = x.detach().clone()
x_after = x.detach().clone()


out = tensor([[[-3.1104, -0.4777, -2.1211],
         [-1.6634, -0.0585, -1.0139],
         [-0.8581, -0.0070, -0.5185],
         [ 0.0281,  0.2477,  0.1658],
         [-0.4765,  0.1084, -0.1927],
         [-0.0140,  0.1900,  0.1065],
         [ 0.0133,  0.1353,  0.1065],
         [ 0.1216,  0.0141,  0.0771],
         [ 0.0057,  0.0227,  0.0102]]], grad_fn=<UnsafeViewBackward0>)

out.size() = torch.Size([1, 9, 3])



Now, comparing x before and after, we see that we get truely different representations:

In [274]:
x_before_col1 = [elem for elem in x_before[0][:,0]]
x_after_col1 = [elem for elem in x_after[0][:,0]]
x_after_col1.reverse()

print(f'before: {"after:":>15} {"diff:":>14}')
for i in range(len(x_before_col1)):
    print(f'{x_before_col1[i]: .3f} {x_after_col1[i]: >15.3f} {x_before_col1[i] - x_after_col1[i]: >15.3f}')


before:          after:          diff:
 3.205          -0.280           3.484
-3.195          -1.152          -2.042
-0.889          -1.069           0.180
 1.705          -0.160           1.866
-1.956          -1.956           0.000
-0.859           1.007          -1.866
-1.069          -0.889          -0.180
 0.284          -1.759           2.042
-0.701           2.783          -3.484


# Misc

In [159]:
spltting_pattern = re.compile(r'(?<=\w)(?=[^\w])|(?<=[^\w])(?=\w)|(?<=[^\w])(?=\s)|(?<=[^\w])(?=[^w])')
foo = 'Hi!! My name is Donald. What is your name? :)'
res = spltting_pattern.split(foo)
print(res)




['Hi', '!', '!', ' ', 'My', ' ', 'name', ' ', 'is', ' ', 'Donald', '.', ' ', 'What', ' ', 'is', ' ', 'your', ' ', 'name', '?', ' ', ':', ')']
